In [1]:
#imports needed for data etl process
import pandas as pd
import numpy as np
import requests

Data Source Extraction

In [2]:
#utilizes the nasa Global landslide catalog
url= 'https://data.nasa.gov/resource/dd9e-wu2v.json?$limit=50000'
#call the actual data
response= requests.get(url).json()

In [3]:
#verify if response from json object was correct
print(response[0])
print(response[1])
print(response[11000])

{'source_name': '3 News', 'source_link': {'url': 'http://www.3news.co.nz/Kakapo-Maggie-dies-in-landslide/tabid/1160/articleID/347074/Default.aspx'}, 'event_id': '6034', 'event_date': '2014-05-20T00:00:00.000', 'event_title': 'Codfish Island', 'event_description': "A landslide on Codfish Island has killed Maggie the kakapo, dealing another blow to the already small population of rare bird. Kakapo Recovery rangers found the bird's body last week buried deep in mud and debris which cut a path down the hill during heavy rain. Manager Deidre Vercoe Scott says data from the adult female's transmitter confirmed the death occurred on May 20.", 'location_description': 'Codfish Island', 'location_accuracy': '5km', 'landslide_category': 'landslide', 'landslide_trigger': 'downpour', 'landslide_size': 'small', 'landslide_setting': 'natural_slope', 'fatality_count': '0', 'injury_count': '0', 'photo_link': {'url': 'http://blogs.agu.org/landslideblog/files/2014/06/14_06-kakapo-3.jpg'}, 'event_import_s

In [4]:
data = []
for item in response:
    try:
        data.append({
            'landslide_id': item.get('event_id'),
            'landslide_date': item.get('event_date'),
            'location_accuracy': item.get('location_accuracy'),
            'category_landslide': item.get('landslide_category'),
            'landslide_trig': item.get('landslide_trigger'),
            'landslide_area': item.get('landslide_size'),
            'landslide_env': item.get('landslide_setting'),
            'fatalities': item.get('fatality_count'),
            'injuries': item.get('injury_count'),
            'country': item.get('country_name'),
            'lng': item.get('longitude'),
            'lat': item.get('latitude')
        })
    except KeyError:
        pass

# Convert the list of dictionaries to a DataFrame
landslide_basedf = pd.DataFrame(data)

In [5]:
#show dataframe
landslide_basedf

,landslide_id,landslide_date,location_accuracy,category_landslide,landslide_trig,landslide_area,landslide_env,fatalities,injuries,country,lng,lat
0,6034,2014-05-20T00:00:00.000,5km,landslide,downpour,small,natural_slope,0,0,New Zealand,167.6337,-46.7748
1,1910,2010-05-29T06:45:00.000,10km,landslide,downpour,medium,unknown,0,0,New Zealand,170.0982,-45.9034
2,6997,2015-06-04T00:00:00.000,5km,landslide,downpour,medium,below_road,0,0,New Zealand,170.6273,-45.8767
3,4941,2013-06-17T00:00:00.000,5km,landslide,downpour,medium,unknown,0,None,New Zealand,170.5863,-45.8727
4,6008,2014-05-01T04:00:00.000,1km,landslide,rain,small,unknown,0,0,New Zealand,170.509,-45.8618
...,...,...,...,...,...,...,...,...,...,...,...,...
11028,8365,2015-09-23T00:00:00.000,1km,landslide,unknown,large,natural_slope,0,0,Iceland,-16.7406,65.0167
11029,6651,2014-07-22T00:00:00.000,1km,landslide,unknown,large,unknown,0,0,Iceland,-16.6849,65.0216
11030,6134,2014-07-10T13:00:00.000,5km,mudslide,rain,small,unknown,0,0,Iceland,-21.4541,65.9794
11031,9686,2017-06-17T13:20:00.000,1km,landslide,freeze_thaw,catastrophic,above_coast,4,9,None,-53.20874578,71.53659933


In [6]:
#begin transforming data through identifying null values
landslide_basedf.isna().sum()

landslide_id             0
landslide_date           0
location_accuracy        2
category_landslide       1
landslide_trig          23
landslide_area           9
landslide_env           69
fatalities            1385
injuries              5674
country               1562
lng                      0
lat                      0
dtype: int64

In [7]:
# drop null values while still maintaing data integrity
transformed_ls_df=landslide_basedf.dropna(axis='index',how='any',subset=['location_accuracy', 'category_landslide', 'landslide_trig', 'landslide_area', 'landslide_env'])

In [8]:
#drop columns with excessive missing data that can't be filled
transformed_ls_df=transformed_ls_df.drop(columns=['country', 'lng','lat'])

In [9]:
#revalidate that correct values are presented, goal is to have no null values present 
transformed_ls_df.isna().sum()

landslide_id             0
landslide_date           0
location_accuracy        0
category_landslide       0
landslide_trig           0
landslide_area           0
landslide_env            0
fatalities            1380
injuries              5665
dtype: int64

In [10]:
#replace null injuires and fatalities values with 0 as it is logical to assume that if no note of these counts  occured, then it didn't pose a presece to them
transformed_ls_df=transformed_ls_df.fillna(value='0')

In [11]:
#revalidate that correct values are presented, goal is to have no null values present 
transformed_ls_df.isna().sum()

landslide_id          0
landslide_date        0
location_accuracy     0
category_landslide    0
landslide_trig        0
landslide_area        0
landslide_env         0
fatalities            0
injuries              0
dtype: int64

In [12]:
#reset index to determine usable rows after null cleansing
transformed_ls_df=transformed_ls_df.reset_index(drop=True)

In [13]:
transformed_ls_df

,landslide_id,landslide_date,location_accuracy,category_landslide,landslide_trig,landslide_area,landslide_env,fatalities,injuries
0,6034,2014-05-20T00:00:00.000,5km,landslide,downpour,small,natural_slope,0,0
1,1910,2010-05-29T06:45:00.000,10km,landslide,downpour,medium,unknown,0,0
2,6997,2015-06-04T00:00:00.000,5km,landslide,downpour,medium,below_road,0,0
3,4941,2013-06-17T00:00:00.000,5km,landslide,downpour,medium,unknown,0,0
4,6008,2014-05-01T04:00:00.000,1km,landslide,rain,small,unknown,0,0
...,...,...,...,...,...,...,...,...,...
10939,8365,2015-09-23T00:00:00.000,1km,landslide,unknown,large,natural_slope,0,0
10940,6651,2014-07-22T00:00:00.000,1km,landslide,unknown,large,unknown,0,0
10941,6134,2014-07-10T13:00:00.000,5km,mudslide,rain,small,unknown,0,0
10942,9686,2017-06-17T13:20:00.000,1km,landslide,freeze_thaw,catastrophic,above_coast,4,9


In [14]:
#drop the time value in landslide_date based on empirical evidence that times are not truthful
transformed_ls_df['landslide_date'] = transformed_ls_df['landslide_date'].str.split('T').str[0] 

In [15]:
transformed_ls_df

,landslide_id,landslide_date,location_accuracy,category_landslide,landslide_trig,landslide_area,landslide_env,fatalities,injuries
0,6034,2014-05-20,5km,landslide,downpour,small,natural_slope,0,0
1,1910,2010-05-29,10km,landslide,downpour,medium,unknown,0,0
2,6997,2015-06-04,5km,landslide,downpour,medium,below_road,0,0
3,4941,2013-06-17,5km,landslide,downpour,medium,unknown,0,0
4,6008,2014-05-01,1km,landslide,rain,small,unknown,0,0
...,...,...,...,...,...,...,...,...,...
10939,8365,2015-09-23,1km,landslide,unknown,large,natural_slope,0,0
10940,6651,2014-07-22,1km,landslide,unknown,large,unknown,0,0
10941,6134,2014-07-10,5km,mudslide,rain,small,unknown,0,0
10942,9686,2017-06-17,1km,landslide,freeze_thaw,catastrophic,above_coast,4,9


In [16]:
#interpret value counts for each column to see how data can be expressed
transformed_ls_df['landslide_trig'].value_counts()

landslide_trig
downpour                   4670
rain                       2555
unknown                    1689
continuous_rain             745
tropical_cyclone            561
snowfall_snowmelt           132
monsoon                     129
mining                       93
earthquake                   89
construction                 79
flooding                     72
no_apparent_trigger          42
freeze_thaw                  41
other                        24
dam_embankment_collapse      11
leaking_pipe                 10
volcano                       1
vibration                     1
Name: count, dtype: int64

In [17]:
transformed_ls_df.dtypes

landslide_id          object
landslide_date        object
location_accuracy     object
category_landslide    object
landslide_trig        object
landslide_area        object
landslide_env         object
fatalities            object
injuries              object
dtype: object

In [18]:
#setting dtypes for each column into appropriate fields as to ensure proper data usability
transformed_ls_df["landslide_id"]=transformed_ls_df["landslide_id"].astype("int16")
transformed_ls_df["landslide_date"]=transformed_ls_df["landslide_date"].astype("datetime64[ns]")
transformed_ls_df["location_accuracy"]=transformed_ls_df["location_accuracy"].astype("category")
transformed_ls_df["category_landslide"]=transformed_ls_df["category_landslide"].astype("category")
transformed_ls_df["landslide_trig"]=transformed_ls_df["landslide_trig"].astype("category")
transformed_ls_df["landslide_area"]=transformed_ls_df["landslide_area"].astype("category")
transformed_ls_df["landslide_env"]=transformed_ls_df["landslide_env"].astype("category")
transformed_ls_df["injuries"]=transformed_ls_df["injuries"].astype("int16")
transformed_ls_df["fatalities"]=transformed_ls_df["fatalities"].astype("int16")

In [19]:
#creating an additional column that serves as a risk section determined as a binary factor, and a plausible target value
transformed_ls_df['risk'] = np.where((transformed_ls_df["injuries"] > 0) | (transformed_ls_df["fatalities"] > 0), True, False)

In [20]:
transformed_ls_df

,landslide_id,landslide_date,location_accuracy,category_landslide,landslide_trig,landslide_area,landslide_env,fatalities,injuries,risk
0,6034,2014-05-20,5km,landslide,downpour,small,natural_slope,0,0,False
1,1910,2010-05-29,10km,landslide,downpour,medium,unknown,0,0,False
2,6997,2015-06-04,5km,landslide,downpour,medium,below_road,0,0,False
3,4941,2013-06-17,5km,landslide,downpour,medium,unknown,0,0,False
4,6008,2014-05-01,1km,landslide,rain,small,unknown,0,0,False
...,...,...,...,...,...,...,...,...,...,...
10939,8365,2015-09-23,1km,landslide,unknown,large,natural_slope,0,0,False
10940,6651,2014-07-22,1km,landslide,unknown,large,unknown,0,0,False
10941,6134,2014-07-10,5km,mudslide,rain,small,unknown,0,0,False
10942,9686,2017-06-17,1km,landslide,freeze_thaw,catastrophic,above_coast,4,9,True


In [21]:
transformed_ls_df['risk'].value_counts()

risk
False    8376
True     2568
Name: count, dtype: int64

In [22]:
#sort dataframe in order of latest occurence 
transformed_ls_df.sort_values(by='landslide_date', inplace=True,ascending= False)

In [23]:
#Load dataframe into a parquet file
transformed_ls_df.to_parquet("CleanedLS.parquet")

In [24]:
#read CleanedLS in to see if file was read properly
test=pd.read_parquet("CleanedLS.parquet")

In [26]:
test

,landslide_id,landslide_date,location_accuracy,category_landslide,landslide_trig,landslide_area,landslide_env,fatalities,injuries,risk
7209,10393,2017-09-28,exact,rock_fall,no_apparent_trigger,large,natural_slope,0,1,True
7210,10392,2017-09-27,exact,rock_fall,no_apparent_trigger,medium,natural_slope,1,1,True
3523,11013,2017-09-26,5km,mudslide,downpour,medium,natural_slope,0,0,False
3945,10971,2017-09-11,25km,landslide,flooding,unknown,unknown,2,0,True
5174,10991,2017-09-10,25km,landslide,rain,medium,natural_slope,0,9,True
...,...,...,...,...,...,...,...,...,...,...
9540,7831,1996-12-26,50km,debris_flow,freeze_thaw,small,above_river,0,0,False
9484,7830,1996-12-26,50km,debris_flow,freeze_thaw,small,above_road,0,0,False
9838,7408,1995-08-14,10km,debris_flow,downpour,large,unknown,0,0,False
10768,7331,1993-06-04,1km,other,rain,medium,deforested_slope,0,0,False
